In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

from keras.models import Model
from keras.layers import Dense,Input
from keras.layers.convolutional import Conv2D, AtrousConvolution2D
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
epochs = 10
range_min = -999
range_max = 999
batch_size = 4 
num_classes = 2
num_train_samples = 1000
num_test_samples = 100

In [3]:
x_train = np.random.randint(low = range_min,high = range_max+1,size = (num_train_samples,2))
y_train = x_train[:,0] * x_train[:,1]
y_train = np.abs(y_train) / y_train
y_train[y_train < 0] = 0

In [4]:
x_test = np.random.randint(low = range_min,high = range_max+1,size = (num_test_samples,2))
y_test = x_test[:,0] * x_test[:,1]
y_test = np.abs(y_test) / y_test
y_test[y_test < 0] = 0

In [5]:
train_data = torch.from_numpy(x_train).type(torch.FloatTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
test_labels = torch.from_numpy(y_test).type(torch.LongTensor)

In [6]:
def keras_to_pyt(km, pm):
    weight_dict = dict()
    for layer in km.layers:
        if type(layer) is keras.layers.convolutional.Conv2D:
            weight_dict[layer.get_config()['name'] + '.weight'] = np.transpose(layer.get_weights()[0], (3, 2, 0, 1))
            weight_dict[layer.get_config()['name'] + '.bias'] = layer.get_weights()[1]
        elif type(layer) is keras.layers.Dense:
            weight_dict[layer.get_config()['name'] + '.weight'] = np.transpose(layer.get_weights()[0], (1, 0))
            weight_dict[layer.get_config()['name'] + '.bias'] = layer.get_weights()[1]
    pyt_state_dict = pm.state_dict()
    for key in pyt_state_dict.keys():
        pyt_state_dict[key] = torch.from_numpy(weight_dict[key])
    pm.load_state_dict(pyt_state_dict)
    return (pm,pyt_state_dict)

In [11]:
class Net_sign(nn.Module):
    def __init__(self):
        super(Net_sign,self).__init__()
        self.fc1 = nn.Linear(2,15)
        self.fc2 = nn.Linear(15,15)
        self.fc3 = nn.Linear(15,2)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = F.softsign(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
    def predict(self,x):
        pred = self.forward(x)
        _,out = torch.max(pred,1)
        return out
        

In [12]:
net = Net_sign()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [13]:
def net_sign(inputs):
    input_data = Input(shape=inputs)
    out = Dense(15,activation = 'softsign',name='fc1')(input_data)
    out = Dense(15,activation = 'softsign',name='fc2')(out)
    out = Dense(2,activation = 'softmax',name='fc3')(out)

    return Model(inputs=input_data,outputs=out)

In [14]:
model_sign = net_sign((2,))
model_sign.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
# model_sign.summary()
model_sign.trainable=False
model_sign.load_weights('../neg_pos.h5')

In [15]:
net,weights = keras_to_pyt(model_sign, net)

In [16]:
pred = net.forward(test_data)
_,indices = torch.max(pred,1)
indices = indices.numpy()
print(indices)
print(test_labels)

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


[0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 0
 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0]
tensor([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 0])


In [18]:
torch.save(net,'model_intial_sign')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sign. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
